<a href="https://colab.research.google.com/github/UthpalaPitawela/Data_Science_Implementations/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
import pandas as pd
import nltk
import numpy as np

import tensorflow as tf

#libraries for pre processing
from bs4 import BeautifulSoup
!pip install contractions
import contractions
import re
import string
#to remove stop words
nltk.download('stopwords')
from nltk.tokenize import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
# custom: removing words from list
stopword_list.remove('not')

#for lemmatization
import spacy
nlp = spacy.load('en',parse=True,tag=True, entity=True)

#libraries for feature extraction
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

#libraries for evaluation metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#libraries for SVM
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score





     |████████████████████████████████| 283 kB 5.7 MB/s 
     |████████████████████████████████| 321 kB 45.0 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85448 sha256=772e16943cd561ff0278803cc8a0f3e4d778c5f724908954a1398ee3160a3336
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#for word embeddings
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

Data load from Google Drive

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1FJYN3RvVIe_zb1XL8w7pbuXRv3S6O38S'
downloaded = drive.CreateFile({'id': file_id})

In [ ]:
# Download the file to a local disk as 'dataset.xlsx'.
downloaded.GetContentFile('dataset.xlsx')

In [ ]:
# Here it is --
!ls -lha dataset.xlsx

-rw-r--r-- 1 root root 232K Sep  2 02:37 dataset.xlsx


In [ ]:
# Now, we can use pandas read_excel after installing the excel importer.
!pip install -q xlrd

import pandas as pd
columns = ['question', 'coarse', 'fine']
df = pd.read_excel('dataset.xlsx', header=None,names=columns)
df

,question,coarse,fine
0,What are the special things we (husband and me...,TTD,TTDSIG
1,What are the companies which organize shark fe...,TTD,TTDOTH
2,Is it safe for female traveller to go alone to...,TGU,TGUHEA
3,What are the best places around Cape Town for ...,TTD,TTDSIG
4,What are the best places to stay for a family ...,ACM,ACMOTH
...,...,...,...
4995,What is the best area to be based for sightsee...,TTD,TTDSIG
4996,What are the good value traditional bars and r...,FOD,FODBAR
4997,What are the hotels near Alicante bus station?,ACM,ACMHOT
4998,Where to stay in La Gomera to mountain biking?,TTD,TTDSPO


Pre processing


*   Remove HTML tags
*   Expand contractions
*   Remove special cases
*   Lowercase all texts
*   Remove stop words
*   Lemmatization









Remove HTML tags

In [ ]:
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


df['html_removed_questions'] = [strip_html_tags(question) for question in df['question']]
df['html_removed_questions'][1]

'What are the companies which organize shark feeding events for scuba divers?'

Expand contractions

In [ ]:
def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text

item = "What are the best tourist excursions when it's raining?";

df['contractions_removed_questions'] = [expand_contractions(question) for question in df['html_removed_questions']]
print(item);
print(expand_contractions(item));


What are the best tourist excursions when it's raining?
What are the best tourist excursions when it is raining?


Remove special cases and punctuation

In [ ]:
# function to remove special characters
def remove_special_characters_punctuation(text):
    # define the pattern to keep
    pat = r'[^a-zA-z.,!?/:;\"\'\s\n]' 
    formattedText = re.sub(pat, '', text)
    text = ''.join([c for c in formattedText if c not in string.punctuation])
    return text
    
item = "What are the best tourist excursions when it's raining?";

df['special_characters_removed_questions'] = [remove_special_characters_punctuation(question) for question in df['contractions_removed_questions']]
print(item);
print(remove_special_characters_punctuation(item));

What are the best tourist excursions when it's raining?
What are the best tourist excursions when its raining


Lower case text

In [ ]:
def lower_case_text(text):
  text = text.strip().lower()
  return text;

item = "Does anyone have the e-mail address for the Ambre resort & spa Mauritius?";

df['lower_cased_questions'] = [lower_case_text(question) for question in df['special_characters_removed_questions']]
print(item);
print(lower_case_text(item));



Does anyone have the e-mail address for the Ambre resort & spa Mauritius?
does anyone have the e-mail address for the ambre resort & spa mauritius?


Remove stop words

In [ ]:
# function to remove stopwords
def remove_stopwords(text):
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    # check in lowercase 
    t = [token for token in tokens if token.lower() not in stopword_list]
    text = ' '.join(t)    
    return text

item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['stop_words_removed_questions'] = [remove_stopwords(question) for question in df['lower_cased_questions']]
print(item);
print(remove_stopwords(item));

What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?
ideal time visit Mauritius year planning get married spend honeymoon ?


Lemmatization

In [ ]:
def get_lem(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text
  
item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['lemmatized_questions'] = [get_lem(question) for question in df['stop_words_removed_questions']]
print(item);
print(get_lem(item));

What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?
what be the ideal time to visit Mauritius in the year as we be plan to get marry in there and spend our honeymoon ?


Create Features

*   POS tags
*   Named Entities
*   Head Word Synonyms
*   Bi-grams
*   Head words





















POS Tags

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def pos_tagging(text):
    words = nltk.word_tokenize(text)
    lst = [ r[1] for r in pos_tag(words)] 
    return ' '.join(lst)


item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['pos_tagged_questions'] = [pos_tagging(question) for question in df['stop_words_removed_questions']]
print(pos_tagging(item));


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
WP VBZ DT JJ NN TO VB NNP IN DT NN IN PRP VBP VBG TO VB VBN IN EX CC VB PRP$ NN .


Named Entities

In [ ]:
def getNamedEntities(text):
  NER_text = NER(text);
  NER_chunk = [];
  for word in NER_text.ents:
    label  = word.label_;
    if label not in NER_chunk:
      NER_chunk.append(word.label_)
  def remove_null(x):
        if '' in x:
            x.remove('')
        return x
  lst = remove_null(NER_chunk)
  return ' '.join(lst)
  
item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['named_entity_questions'] = [getNamedEntities(question) for question in df['stop_words_removed_questions']]
print(getNamedEntities(item));


GPE DATE


Count vectorizer (BOW)

In [ ]:
def get_count_vect(documents):
    vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)#, stop_words=stopwords.words('english')
    X = vectorizer.fit_transform(documents).toarray()
    print(X.shape)
    return X

get_count_vect(df['stop_words_removed_questions'])

(5000, 1072)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Head word Synonyms

In [ ]:
from nltk.corpus import wordnet 
nltk.download('wordnet')
def get_synonyms(words):
    all_synonyms = []
    for word in words.split(' '):
        synonyms = []

        for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())

        synonyms = list(set(synonyms))
        all_synonyms += synonyms
        
    return all_synonyms


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

head_words_synonym_vectorizer = CountVectorizer(tokenizer = get_synonyms,max_features=100,stop_words=stopwords.words('english'))
head_words_synonym_vector = head_words_synonym_vectorizer.fit_transform(df['stop_words_removed_questions'].values).toarray()
print(head_words_synonym_vector)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'tween", '1', '1000', '500', 'A', 'AM', 'AN', 'AS', 'Am', 'American_Samoa', 'Artium_Magister', 'As', 'Associate_in_Nursing', 'At', 'Bay_State', 'Be', 'Beaver_State', 'D', 'DO', 'DOE', 'Department_of_Energy', 'Doctor_of_Osteopathy', 'Don', 'Don_River', 'Down', 'Eastern_Samoa', 'Energy', 'Energy_Department', 'Evergreen_State', 'G', 'HA', 'He', 'Hera', 'Here', 'Hoosier_State', 'I', 'IN', 'ISN', 'IT', 'In', 'Indiana', 'International_Relations_and_Security_Network', 'John_L._H._Down', 'K', "KO'd", 'Lapp', 'Lapplander', 'M', 'MA', 'MB', 'ME', 'MT', 'Maine', 'Massachusetts', 'Master_of_Arts', 'MiB', 'More', 'No', 'North_Korean_won', 'O', 'OR', 'Old_Colony', 'Oregon', 'Pine_Tree_State', 'Ra', 'Re', 'S', 'Saame', 'Saami', 'Same', 'Sami', 'Shan', 'Sir_Thomas_More', 'South_Korean_won', 'T', 'Tai_Long',

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Head Words

In [ ]:
# Head word tokenizer
import spacy
nlp = spacy.load("en_core_web_sm")
def head_word_tokenizer(text):
    head_words = []
    for token in nlp(text):
        if token.dep_ == "nsubj" or token.dep_ == "nsubjpass":
            head_words.append(token.text)
    return head_words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

head_words_vectorizer = CountVectorizer(tokenizer = head_word_tokenizer,max_features=100,stop_words=stopwords.words('english'))
head_words_vector = head_words_vectorizer.fit_transform(df["stop_words_removed_questions"].values).toarray()
print(head_words_vector)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Bi-grams

In [ ]:
def bigram(text):
    new_words = ""
    pre_word = None
    for word in text.strip().split(' '):
        
        if pre_word is not None:
            new_words += "{}{} ".format(pre_word, word)
        pre_word = word
    return new_words[:-1]

item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['bigram_questions'] = [bigram(question) for question in df['stop_words_removed_questions']]

print(bigram(item))

Whatis isthe theideal idealtime timeto tovisit visitMauritius Mauritiusin inthe theyear yearas aswe weare areplanning planningto toget getmarried marriedin inthere thereand andspend spendour ourhoneymoon?


Define course class values

In [ ]:
def get_class_values(feature_set='coarse'):
    le = LabelEncoder()
    encoded_classes = le.fit_transform(df[feature_set])
    return encoded_classes

Define evaluation metrics

In [ ]:
def PRC_matrics(y_test, prediction):
    # prediction
    precision = precision_score(y_test, prediction, labels=np.unique(prediction), average='micro')*100
    print('Precision: %.3f' % precision)

    # recall
    recall = recall_score(y_test, prediction, labels=np.unique(prediction), average='micro')*100
    print('Recall: %.3f' % recall)
    
    # score
    f1_score = 2 * (precision * recall) / (precision + recall)
    print('F-Measure: %.3f' % f1_score)
    
    
    acc = accuracy_score(y_test, prediction)*100
    print('Accuracy score: %.3f' % acc)
    
    
    cm = confusion_matrix(y_test, prediction)
    print("\nConfustion matrix: \n{}".format(cm))
    
    return precision, recall, f1_score, acc, cm

Define SVM

In [ ]:
def train_with_svm(XX, y):
    best_prediction = None
    best_test = None
    best_accuracy = 0
    cv = KFold(n_splits=10, random_state=42, shuffle=True)
    fold = 0
    accuracies = []
    for train_index, test_index in cv.split(XX):
        fold += 1
        X_train, X_test = XX[train_index], XX[test_index]
        y_train, y_test = y[train_index], y[test_index]

        SVM = SVC(C=1.0, kernel='linear', degree=4, gamma='auto')
        SVM.fit(X_train,y_train)
        predictions_SVM1 = SVM.predict(X_test)
        acc = accuracy_score(predictions_SVM1, y_test)*100
        if best_accuracy < acc:
            best_accuracy = acc
            best_prediction = predictions_SVM1
            best_test = y_test
            best_model = SVM
        accuracies.append(acc)
        print("Fold - {} - {} - {:.2f}".format(fold, "Accuracy-> ",acc))

    print("Mean Accuracy {:.2f} \nStd Accuracy {:.2f}\n\n".format(np.mean(accuracies), np.std(accuracies)))
    
    print("Best accuracy : {}".format(best_accuracy))
    PRC_matrics(best_test, best_prediction)

**Question 01: A traditional ML classifier s.a. SVM or Logistic Regression with at least 5  of the features mentioned in the paper**

Selected Features

*   POS tags
*   Named Entities
*   Head words
*   Head Word Synonyms
*   Bi-grams

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import coo_matrix, csr_matrix, hstack


coarse = get_class_values('coarse')

X_lem = df['lemmatized_questions']
tfidf_lem = TfidfVectorizer(max_features=5000)
tfidf_lem.fit(X_lem)

X_pos = df['pos_tagged_questions']

X_ne = df['named_entity_questions']

X_bigram = df['bigram_questions']


XX = csr_matrix(hstack([tfidf_lem.transform(X_lem) ,get_count_vect(X_pos), get_count_vect(X_ne), get_count_vect(X_bigram),
                        head_words_vector,head_words_synonym_vector]))
XX.shape

(5000, 22)
(5000, 13)
(5000, 344)


(5000, 5561)

Feature Selection

1. Select k best features

In [ ]:

from sklearn.feature_selection import SelectKBest, chi2

X_new = SelectKBest(chi2, k=3500).fit_transform(XX, coarse)
X_new.shape

(5000, 3500)

2. Remove features considering a specified threshold

In [ ]:
from sklearn.feature_selection import VarianceThreshold
X_new_var = VarianceThreshold(threshold=(0.01)).fit_transform(XX)
X_new_var.shape

(5000, 132)

Training using the defined SVM

1. Train with coarse classes

In [ ]:
train_with_svm(XX, get_class_values('coarse'))

Fold - 1 - Accuracy->  - 80.20
Fold - 2 - Accuracy->  - 78.00
Fold - 3 - Accuracy->  - 78.80
Fold - 4 - Accuracy->  - 75.80
Fold - 5 - Accuracy->  - 76.40
Fold - 6 - Accuracy->  - 78.40
Fold - 7 - Accuracy->  - 81.00
Fold - 8 - Accuracy->  - 77.20
Fold - 9 - Accuracy->  - 80.00
Fold - 10 - Accuracy->  - 77.60
Mean Accuracy 78.34 
Std Accuracy 1.60


Best accuracy : 81.0
Precision: 81.000
Recall: 81.162
F-Measure: 81.081
Accuracy score: 81.000

Confustion matrix: 
[[ 0  0  1  0  0  0  0  0]
 [ 0 71  0  0  4  0  8  0]
 [ 0  0 14  2  1  1  6  0]
 [ 0  1  0 40  3  1  5  0]
 [ 0  5  1  2 83  6 14  0]
 [ 0  2  1  0  5 97  4  0]
 [ 0  5  3  0  7  5 92  0]
 [ 0  0  0  0  1  0  1  8]]


2. Train with fine classes

In [ ]:
train_with_svm(XX, get_class_values('fine'))

Fold - 1 - Accuracy->  - 53.00
Fold - 2 - Accuracy->  - 56.60
Fold - 3 - Accuracy->  - 50.80
Fold - 4 - Accuracy->  - 51.60
Fold - 5 - Accuracy->  - 49.40
Fold - 6 - Accuracy->  - 49.80
Fold - 7 - Accuracy->  - 55.60
Fold - 8 - Accuracy->  - 49.80
Fold - 9 - Accuracy->  - 53.20
Fold - 10 - Accuracy->  - 51.20
Mean Accuracy 52.10 
Std Accuracy 2.35


Best accuracy : 56.599999999999994
Precision: 56.600
Recall: 58.714
F-Measure: 57.637
Accuracy score: 56.600

Confustion matrix: 
[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ... 11  1  0]
 [ 0  0  0 ...  1  1  0]
 [ 0  0  0 ...  0  0  1]]


**Question 02: 2. A traditional ML classifier s.a. SVM or Logistic Regression with word embedding features s.a. word2vec or fasttext.**


Word Embeddings: Using fasttext

In [ ]:
def get_embedding(sentence):    
    embedding=ft.get_sentence_vector(sentence)
    return embedding

In [ ]:
X_embedding = np.array([get_embedding(question) for question in df['stop_words_removed_questions'].values])
le = LabelEncoder()

Train with SVM for coarse classes

In [ ]:
train_with_svm(X_embedding, get_class_values('coarse'))

Fold - 1 - Accuracy->  - 77.80
Fold - 2 - Accuracy->  - 78.80
Fold - 3 - Accuracy->  - 81.60
Fold - 4 - Accuracy->  - 77.80
Fold - 5 - Accuracy->  - 78.60
Fold - 6 - Accuracy->  - 78.40
Fold - 7 - Accuracy->  - 80.20
Fold - 8 - Accuracy->  - 76.00
Fold - 9 - Accuracy->  - 78.40
Fold - 10 - Accuracy->  - 77.80
Mean Accuracy 78.54 
Std Accuracy 1.43


Best accuracy : 81.6
Precision: 81.600
Recall: 81.600
F-Measure: 81.600
Accuracy score: 81.600

Confustion matrix: 
[[ 50   0   5   2   0   3   0]
 [  0  12   0   1   0   4   0]
 [  1   0  41   2   0   6   0]
 [  5   0   0  92  10  16   0]
 [  1   0   0   5 100   6   0]
 [  3   0   2  11   7  87   0]
 [  0   0   0   0   0   2  26]]


Train with SVM for fine classes

In [ ]:
train_with_svm(X_embedding, get_class_values('fine'))

Fold - 1 - Accuracy->  - 53.40
Fold - 2 - Accuracy->  - 55.80
Fold - 3 - Accuracy->  - 57.00
Fold - 4 - Accuracy->  - 54.40
Fold - 5 - Accuracy->  - 51.60
Fold - 6 - Accuracy->  - 54.00
Fold - 7 - Accuracy->  - 58.00
Fold - 8 - Accuracy->  - 51.00
Fold - 9 - Accuracy->  - 55.60
Fold - 10 - Accuracy->  - 52.40
Mean Accuracy 54.32 
Std Accuracy 2.18


Best accuracy : 57.99999999999999
Precision: 58.000
Recall: 65.909
F-Measure: 61.702
Accuracy score: 58.000

Confustion matrix: 
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 6 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


**Question 03: A NN classifier s.a. an LSTM for classification**

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 25
# This is fixed.
EMBEDDING_DIM = 160
epochs = 10
batch_size = 64

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ')
tokenizer.fit_on_texts(df['question'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 5445 unique tokens.


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(df['question'].values)
print(X[0])
features = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', features.shape)


def get_class_labels(_type='coarse'):
    y = pd.get_dummies(df[_type]).values
    print('Shape of label tensor:', y.shape)
    return y

classes = get_class_labels('coarse')

[254, 63, 1694, 19, 6, 304, 67]
Shape of data tensor: (5000, 25)
Shape of label tensor: (5000, 10)


In [ ]:
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras import Sequential

#Define the lstm model
def get_lstm_model(X, y, verbose=0):
    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
    #0.2 dropout rate is used
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
    #softmax is used as the activation function
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    if verbose == 1:
        print(model.summary())
    return model

get_lstm_model(features, get_class_labels('fine'), 1)

Shape of label tensor: (5000, 79)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 25, 160)           800000    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 25, 160)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 196)               279888    
_________________________________________________________________
dense_6 (Dense)              (None, 79)                15563     
Total params: 1,095,451
Trainable params: 1,095,451
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from sklearn.model_selection import train_test_split

#Split train and test as train: 80% and test : 20%
X_train, X_test, Y_train, Y_test = train_test_split(features,classes, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4000, 25) (4000, 10)
(1000, 25) (1000, 10)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold

epochs = 10
batch_size = 64

def train_LSTM(X, y):
    best_prediction = None
    best_test = None
    best_accuracy = 0
    
    #K fold cross validation is used with 10 splits
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    fold = 0
    accuracies = []
    for train_index, test_index in cv.split(X):
        fold += 1
        print("FOLD {}".format(fold))
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = get_lstm_model(X, y, 0)

        hist = model.fit(X_train, y_train, verbose=0, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
        predictions = model.predict(X_test)
        for item in hist.history.items():
            mean_val = np.mean(item[1])
            if item[0] == 'accuracy':
                acc = mean_val
            print("Mean {} : {}".format(item[0], mean_val))
        
        
        print("\n")
        if best_accuracy < acc:
            best_accuracy = acc
            best_prediction = predictions
            best_test = y_test
            
    fine_pred = [np.argmax(p) for p in best_prediction]
    fine_gt = [np.argmax(p) for p in best_test]
    PRC_matrics(fine_pred, fine_gt)

In [ ]:
train_LSTM(features, get_class_labels('coarse'))


Shape of label tensor: (5000, 10)
FOLD 1
Mean loss : 0.5125426459126174
Mean accuracy : 0.8130555674433708
Mean val_loss : 1.218598186969757
Mean val_accuracy : 0.5986111266538501


FOLD 2
Mean loss : 0.4642193594740497
Mean accuracy : 0.8363237414095137
Mean val_loss : 1.096541358364953
Mean val_accuracy : 0.6486419720782174


FOLD 3
Mean loss : 0.5228382371779945
Mean accuracy : 0.8091357946395874
Mean val_loss : 1.2897752655877008
Mean val_accuracy : 0.6071604995263947


FOLD 4
Mean loss : 0.7397257698078951
Mean accuracy : 0.7301646073659261
Mean val_loss : 1.1563325822353363
Mean val_accuracy : 0.578518512348334


FOLD 5
Mean loss : 0.6294968032411167
Mean accuracy : 0.7709700231041227
Mean val_loss : 1.2383652925491333
Mean val_accuracy : 0.5698412539703506


FOLD 6
Mean loss : 0.5167178735136986
Mean accuracy : 0.8127297643158171
Mean val_loss : 1.1949146389961243
Mean val_accuracy : 0.6782716049088372


FOLD 7
Mean loss : 0.5866294189223221
Mean accuracy : 0.7880070507526398
Me

In [ ]:
train_LSTM(features, get_class_labels('fine'))

Shape of label tensor: (5000, 79)
FOLD 1
Mean loss : 1.7968821823596954
Mean accuracy : 0.5749135829508305
Mean val_loss : 2.7323614835739134
Mean val_accuracy : 0.32822222271934154


FOLD 2
Mean loss : 1.691644588112831
Mean accuracy : 0.6017530843615532
Mean val_loss : 2.672770953178406
Mean val_accuracy : 0.40888888835906984


FOLD 3
Mean loss : 1.8120527505874633
Mean accuracy : 0.5691851913928986
Mean val_loss : 2.6035063743591307
Mean val_accuracy : 0.37866666400805116


FOLD 4
Mean loss : 1.7629392340779304
Mean accuracy : 0.5853086426854134
Mean val_loss : 2.7090800523757936
Mean val_accuracy : 0.3919999999925494


FOLD 5
Mean loss : 1.7554625779390336
Mean accuracy : 0.5830617241561413
Mean val_loss : 2.6617604732513427
Mean val_accuracy : 0.37244444992393255


FOLD 6
Mean loss : 1.84823559390174
Mean accuracy : 0.5613991792003313
Mean val_loss : 2.6430780622694225
Mean val_accuracy : 0.3713580262329843


FOLD 7
Mean loss : 2.0450053587555885
Mean accuracy : 0.5133950626477599